<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/Python/%D0%92%D1%82%D0%BE%D1%80%D0%BE%D0%B9_%D0%BA%D0%B5%D0%B9%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2 кейс

**Соберите данные из Google Sheets и напишите функцию `generate_report`, которая принимает три списка (данные с каждого листа в Google Sheets) и сохраняет отчет в файл `student_debt_report.txt`**

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [6]:
!wget https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json

!wget https://gist.github.com/Vs8th/39c5deed0f5539d781f00328f7fd4fe0/raw/result.txt

--2024-06-17 06:32:03--  https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.github.com (gist.github.com)... 20.27.177.113
Connecting to gist.github.com (gist.github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json [following]
--2024-06-17 06:32:03--  https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2358 (2.3K) [text/plain]
Saving to: ‘creds.json’

creds.json          100%[===================>]   2.30K  --.-KB/s    in 0s      

2024-06-17 06:32:04 (32.0 MB/s) - ‘creds.json’ saved 

Чтобы посмотреть как выглядят сами гугл таблицы в оригинале - можете перейти по [ссылке](https://docs.google.com/spreadsheets/d/1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U/edit?usp=sharing) и изучить.

Как примерно должен выглядеть результат:

In [7]:
with open('result.txt') as f:
  print(f.read())

Студент Иванов У.У. - долг 100000 рублей
Студент Петров Е.Е. - долг 250000 рублей
Студент Сидоров И.И. - долг 10000 рублей


In [59]:
#@title Если возникнут трудности с определением `scope`, подсказка:

scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

In [78]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np

In [61]:
# Указываем необходимые права доступа к таблицам
scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

# Загружаем ключи аутентификации из файла json
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)

# Авторизуемся в Google Sheets API
client = gspread.authorize(creds)

**Примечание**

Считать должников необходимо на 1 марта 2023 года. То есть определяя количество просроченных платежей, мы определяем их количество не на настоящую дату, а на **1 марта 2023 года**. А периоды внесения платежей для всех студентов одинаковы - **6 месяцев**.

То есть, если ожидаемый платеж должен был пройти 3 февраля 2022 года, то к 1 марта 2023 студент просрочил уже три платежа (3 февраля 2022, 3 августа 2022 и 3 февраля 2023).

**Решение**

Напишите свое решение ниже

In [ ]:
def generate_report(sheet1, sheet2, sheet3):

  ...



## Решение ##
Так как входных параметров для запуска функции **generate_report()** в задании почему-то не предусмотрено, пойдем другим путем и будем получать данные с лисов при помощи функции **client.open_by_url()**, благо ссылка на базовый документ у нас имеется.

In [63]:
url = f"https://docs.google.com/spreadsheets/d/1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U/edit?usp=sharing"
calculation_date = '2023-03-01' # Дата, на которую будем считать должников

In [202]:
def debt_calc(last_payment_date, onetime_payment, left_to_pay, current_date = calculation_date):
  # Функция для расчета рамера задолженности с учетом остатка по оплате
  debt = ((pd.to_datetime(current_date) - last_payment_date - np.timedelta64(1, 'D')) // np.timedelta64(6, 'M')) * onetime_payment
  return debt if debt <= left_to_pay else left_to_pay

In [194]:
def generate_report(client, url, calculation_date):

  # Читаем содержиме листов
  sheet1_data = client.open_by_url(url).worksheet("Лист1").get_all_records()
  sheet2_data = client.open_by_url(url).worksheet("Лист2").get_all_records()
  sheet3_data = client.open_by_url(url).worksheet("Лист3").get_all_records()

  # Преобразуем данные в DataFrame
  s1 = pd.DataFrame.from_dict(sheet1_data).set_index('student_id')
  s2 = pd.DataFrame.from_dict(sheet2_data).set_index('student_id')
  s3 = pd.DataFrame.from_dict(sheet3_data).set_index('student_id')

  # Объединяем данные из 3 df в одну плоскую таблицу
  data = s1.join(s2, how='outer').join(s3, how='outer')

  # Прведем поля с датами к правильному типу
  data['last_payment_date'] = data['last_payment_date'].apply(pd.to_datetime, dayfirst=True)
  data['expected_payment_date'] = data['expected_payment_date'].apply(pd.to_datetime, dayfirst=True)

  # Фильтруем данные, оставив только должников
  data = data.query("installment	 == 'Y' and expected_payment_date < @calculation_date")[:]

  # Размер долга в для случая платежа раз 6 месяцев вариант 1

  data['debt'] = np.vectorize(debt_calc)(data['last_payment_date'], data['one-time_payment'], data['left_to_pay'])

  return data[['student_name', 'debt']]

In [195]:
# Считаем размеры задолженностей и сохраняем в DataFrame
report_data = generate_report(client=client, url=url, calculation_date=calculation_date)

In [197]:
# Пишем информацию о должниках в файл student_debt_report.txt в нужном формате
with open('student_debt_report.txt', 'w') as f:
  for i in report_data.index[:-1]:
    f.write(f"Студент {report_data.loc[i, 'student_name']} - долг {int(report_data.loc[i, 'debt'])} рублей\n")
  f.write(f"Студент {report_data.loc[report_data.index[-1], 'student_name']} - долг {int(report_data.loc[report_data.index[-1], 'debt'])} рублей")

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [199]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt

import pandas as pd

user_answer = pd.read_csv('student_debt_report.txt')
correct_answer = pd.read_csv('student_debt.txt')

--2024-06-17 13:31:50--  https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.github.com (gist.github.com)... 20.27.177.113
Connecting to gist.github.com (gist.github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt [following]
--2024-06-17 13:31:50--  https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11325 (11K) [text/plain]
Saving to: ‘student_debt.txt’

student_debt.txt    100%[===================>]  11.06K  --.-KB/s    in 0.001s  

2024-06-17 13:31:51 (16.0 MB/

In [200]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!
